## G. Deshpande, A. Batliner, and B. W. Schuller, “Ai-based human audio processing for covid-19: A comprehensive overview,” Pattern recognition, vol. 122, p. 108289, 2022.


In [ ]:
import pandas as pd
import numpy as np

import os
import sys

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Paths for data.
Ravdess = "/content/drive/My Drive/PROJECT/Audio Data/RAVDESS/audio_speech_actors_01-24/"
Crema = "/content/drive/My Drive/PROJECT/Audio Data/CREMA/AudioWAV/"
Tess = "/content/drive/My Drive/PROJECT/Audio Data/TESS/TESS Toronto emotional speech set data/"
Savee = "/content/drive/My Drive/PROJECT/Audio Data/SAVEE/ALL/"

In [ ]:
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + dir + '/' + file)

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

<ipython-input-10-a0808166368c>:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)


,Emotions,Path
0,neutral,/content/drive/My Drive/PROJECT/Audio Data/RAV...
1,neutral,/content/drive/My Drive/PROJECT/Audio Data/RAV...
2,neutral,/content/drive/My Drive/PROJECT/Audio Data/RAV...
3,neutral,/content/drive/My Drive/PROJECT/Audio Data/RAV...
4,happy,/content/drive/My Drive/PROJECT/Audio Data/RAV...


In [ ]:
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directory_list:
    # storing file paths
    file_path.append(Crema + file)
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df.head()

,Emotions,Path
0,angry,/content/drive/My Drive/PROJECT/Audio Data/CRE...
1,neutral,/content/drive/My Drive/PROJECT/Audio Data/CRE...
2,fear,/content/drive/My Drive/PROJECT/Audio Data/CRE...
3,neutral,/content/drive/My Drive/PROJECT/Audio Data/CRE...
4,sad,/content/drive/My Drive/PROJECT/Audio Data/CRE...


In [ ]:
tess_directory_list = os.listdir(Tess)

file_emotion = []
file_path = []

for dir in tess_directory_list:
    directories = os.listdir(Tess + dir)
    for file in directories:
        part = file.split('.')[0]
        part = part.split('_')[2]
        if part=='ps':
            file_emotion.append('surprise')
        else:
            file_emotion.append(part)
        file_path.append(Tess + dir + '/' + file)

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Tess_df = pd.concat([emotion_df, path_df], axis=1)
Tess_df.head()

,Emotions,Path
0,disgust,/content/drive/My Drive/PROJECT/Audio Data/TES...
1,disgust,/content/drive/My Drive/PROJECT/Audio Data/TES...
2,disgust,/content/drive/My Drive/PROJECT/Audio Data/TES...
3,disgust,/content/drive/My Drive/PROJECT/Audio Data/TES...
4,disgust,/content/drive/My Drive/PROJECT/Audio Data/TES...


In [ ]:
savee_directory_list = os.listdir(Savee)

file_emotion = []
file_path = []

for file in savee_directory_list:
    file_path.append(Savee + file)
    part = file.split('_')[1]
    ele = part[:-6]
    if ele=='a':
        file_emotion.append('angry')
    elif ele=='d':
        file_emotion.append('disgust')
    elif ele=='f':
        file_emotion.append('fear')
    elif ele=='h':
        file_emotion.append('happy')
    elif ele=='n':
        file_emotion.append('neutral')
    elif ele=='sa':
        file_emotion.append('sad')
    else:
        file_emotion.append('surprise')

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Savee_df = pd.concat([emotion_df, path_df], axis=1)
Savee_df.head()

,Emotions,Path
0,angry,/content/drive/My Drive/PROJECT/Audio Data/SAV...
1,angry,/content/drive/My Drive/PROJECT/Audio Data/SAV...
2,angry,/content/drive/My Drive/PROJECT/Audio Data/SAV...
3,angry,/content/drive/My Drive/PROJECT/Audio Data/SAV...
4,angry,/content/drive/My Drive/PROJECT/Audio Data/SAV...


In [ ]:
# creating Dataframe using all the 4 dataframes we created so far.
data_path = pd.concat([Ravdess_df, Crema_df, Tess_df, Savee_df], axis = 0)
data_path.to_csv("data_path.csv",index=False)
data_path.head()

,Emotions,Path
0,neutral,/content/drive/My Drive/PROJECT/Audio Data/RAV...
1,neutral,/content/drive/My Drive/PROJECT/Audio Data/RAV...
2,neutral,/content/drive/My Drive/PROJECT/Audio Data/RAV...
3,neutral,/content/drive/My Drive/PROJECT/Audio Data/RAV...
4,happy,/content/drive/My Drive/PROJECT/Audio Data/RAV...


In [ ]:
def preprocess(file):
    files_ds = tf.data.Dataset.from_tensor_slices(file)
    output_ds = files_ds.map(get_waveform_label,num_parallel_calls=AUTO)
    output_ds = output_ds.map(get_spectrogram_label,num_parallel_calls=AUTO)
    return output_ds

train_ds = spectrogram_ds
val_ds = preprocess(val_ds)
test_ds = preprocess(test_ds)

batch_size = 64
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

train_ds = train_ds.cache().prefetch(AUTO)
val_ds = val_ds.cache().prefetch(AUTO)

In [ ]:
for spectrogram,_ in spectrogram_ds.take(1):
    input_shape = spectrogram.shape

num_labels = len(labels)
norm_layer = preprocessing.Normalization()
norm_layer.adapt(spectrogram_ds.map(lambda x, _: x))

print(input_shape)
print(num_labels)

In [ ]:
model = Sequential([
   Input(shape=input_shape), preprocessing.Resizing(32, 32), norm_layer,
   Conv2D(32,3, activation='relu'),
   Conv2D(64,3, activation='relu'),
   MaxPool2D(),
   Dropout(0.5),
   Conv2D(128,7, activation='relu'),
   Conv2D(256,7, activation='relu'),
   MaxPool2D(),
   Dropout(0.5),
   Flatten(),
   Dense(128, activation='relu'),
   Dropout(0.2),
   Dense(16, activation='relu'),
   Dense(num_labels),
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

In [1]:
model = Sequential([
   Input(shape=input_shape), preprocessing.Resizing(32, 32), norm_layer,
   Conv2D(32,3, activation='relu'),
   Conv2D(64,3, activation='relu'),
   MaxPool2D(),
   Dropout(0.5),
   Conv2D(128,7, activation='relu'),
   Conv2D(256,7, activation='relu'),
   MaxPool2D(),
   Dropout(0.5),
   Flatten(),
   Dense(128, activation='relu'),
   Dropout(0.2),
   Dense(16, activation='relu'),
   Dense(num_labels),
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 32, 32, 1)         0         
_________________________________________________________________
normalization (Normalization (None, 32, 32, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         

In [2]:
his = model.fit(train_ds, epochs=30, validation_data=val_ds)


Epoch 1/30
3/3 [==============================] - 12s 3s/step - loss: 5.0686 - accuracy: 0.0651 - val_loss: 1.4357 - val_accuracy: 0.3095
Epoch 2/30
3/3 [==============================] - 1s 480ms/step - loss: 2.2736 - accuracy: 0.3955 - val_loss: 0.5964 - val_accuracy: 0.0000e+00
Epoch 3/30
3/3 [==============================] - 1s 473ms/step - loss: 1.0231 - accuracy: 0.1407 - val_loss: 0.6277 - val_accuracy: 0.0476
Epoch 4/30
3/3 [==============================] - 1s 470ms/step - loss: 1.1536 - accuracy: 0.2687 - val_loss: 0.5604 - val_accuracy: 0.4048
Epoch 5/30
3/3 [==============================] - 1s 475ms/step - loss: 0.8662 - accuracy: 0.2586 - val_loss: 0.6728 - val_accuracy: 0.3571
Epoch 6/30
3/3 [==============================] - 1s 469ms/step - loss: 0.9130 - accuracy: 0.3541 - val_loss: 0.5654 - val_accuracy: 0.3333
Epoch 7/30
3/3 [==============================] - 1s 463ms/step - loss: 0.6733 - accuracy: 0.3825 - val_loss: 0.6630 - val_accuracy: 0.3095
Epoch 8/30
3/3 [=